## Import library

In [1]:
import sys
sys.path.insert(
    1,
    '/home/dtd/Documents/interpretable_machine_learning/Source Code/my_work/lib'
)

import data_load
import numpy as np
import pandas as pd
import logging
import dowhy.datasets
import dowhy
import propensity_score_estimator as pse
import incremental_ps_score_estimator as ipse
import math
import timeit
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import utils 
from tempfile import TemporaryFile

# save numpy array as npy file
from numpy import asarray
from numpy import save

from dowhy import CausalModel
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import sem
from dowhy import CausalModel
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from math import sqrt
from econml.dml import LinearDMLCateEstimator
from sklearn.linear_model import LassoCV
from econml.inference import BootstrapInference
from econml.dml import SparseLinearDMLCateEstimator
from sklearn.linear_model import LassoCV, ElasticNetCV
from sklearn.ensemble import GradientBoostingRegressor
from mlens.ensemble import SuperLearner
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from geneticalgorithm import geneticalgorithm as ga

/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
[MLENS] backend: threading


In [4]:
tf.__version__

'2.2.0'

In [5]:
treatment = 't'
outcome = 'yf'
col =  ["t", "yf", "ycf", "mu0", "mu1" ]
cov = ["x" + str(i) for i in range(1,26)]
col = col + cov
features = cov + ["t"]

## Optimization with Adam and original function

In [6]:
def incre_ps(delta, data):
    q1 = (delta * data['p1']) / (delta * data['p1'] + data['p0'])
    q1 = tf.math.abs(q1)
    a0 = (1-q1)*data['w0']*(data['cf0'] - data[outcome])
    a1 = q1*data['w1']*(data['cf1'] - data[outcome])    
    influence = a1 - a0
    return tf.reduce_mean(influence)

def optimization(data):
    threhold = tf.constant([0.001])
    delta = tf.Variable(100., trainable = True)
    true_effect = np.mean(data['mu1'] - data['mu0'])
    
    for i in range(80000):
        with tf.GradientTape() as tape:
            influence = incre_ps(delta, data)
            loss = tf.math.abs(true_effect - influence)
            d_delta = tape.gradient(loss, delta)
            opt = tf.keras.optimizers.Adam(learning_rate=0.001)
            opt.apply_gradients(zip([d_delta], [delta]))
            ## early stopping
            if tf.math.less(loss, threhold):
                print("The performance reach MAE: 0.001. Cancelling the training at step {}".format(i))
                break
    return delta, loss

### Optimize in train data

In [5]:

PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.train.npz"
PATH_TEST = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.test.npz"

delta_seq = []
losses = []
for index_ in tqdm(range(1, 101)): 
    data = utils.load_data(PATH_TRAIN, index_)
    ## Fit treatment
    model_t = LogisticRegression()
    model_t.fit(data[cov], data[treatment])

    ## Fit outcome
    model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    model_y.fit(data[features], data[outcome])

    data['p1'] = model_t.predict_proba(data[cov])[:,1]
    data['p0'] = 1 - data['p1']

    ## Compute counterfactual outcome with no treatment
    data_pos = data.copy()
    data_pos[treatment] = 1
    data['cf1'] = model_y.predict(data_pos[features])

    ## Compute counterfactual outcome with treatment
    data_neg = data.copy()
    data_neg[treatment] = 0
    data['cf0'] = model_y.predict(data_neg[features])

    data['ips_weight'] = (data[treatment] / data['p1'] + (1 - data[treatment]) /
                          (1 - data['p1']))
    
    data['w0'] = data['ips_weight']*data[treatment]
    data['w1'] = data['ips_weight']*(1 - data[treatment])
    
    delta, loss = optimization(data)
    delta_r = delta.numpy()
    delta_seq.append(delta_r)
    losses.append(loss.numpy())
    print("Data index {}. Delta {:.2f}. Loss {:.2f}".format(index_, delta_r, loss.numpy()))

  0%|          | 0/100 [00:13<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
np.mean(losses)

In [ ]:
data = asarray(delta_seq)
save('delta.npy', delta_seq)

## Optimization with list of delta

In [14]:
def incre_ps(delta, data):
    q1 = (delta * data['p1']) / (delta * data['p1'] + data['p0'])
    q1 = tf.math.abs(q1)
    a0 = (1-q1)*data['w0']*(data['cf0'] - data[outcome])
    a1 = q1*data['w1']*(data['cf1'] - data[outcome])    
    influence = a1 - a0
    return tf.reduce_mean(influence)

def optimization(data):
    threhold = tf.constant([0.001])
    '''
    delta = tf.Variable(
        tf.random.uniform([data.shape[0],], 
                          minval=1, 
                          maxval=100, 
                          dtype=tf.dtypes.float32), 
                          trainable = True)
    '''
    delta = tf.Variable(tf.random.normal(
        [data.shape[0],], 
        mean=0.0, 
        stddev=1.0, 
        dtype=tf.dtypes.float32, 
        seed=1, 
        name='delta'
    ), trainable = True)
    
    true_effect = np.mean(data['mu1'] - data['mu0'])
    
    for i in range(50000):
        with tf.GradientTape() as tape:
            influence = incre_ps(delta, data)
            loss = tf.math.abs(true_effect - influence)
            d_delta = tape.gradient(loss, delta)
            opt = tf.keras.optimizers.Adam(learning_rate=0.001)
            opt.apply_gradients(zip([d_delta], [delta]))
            ## early stopping
            if tf.math.less(loss, threhold):
                print("The performance reach MAE: 0.001. Cancelling the training at step {}".format(i))
                break
    return delta, loss

### Optimize in train data

In [18]:
PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.train.npz"
PATH_TEST = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.test.npz"

delta_seq = []
for index_ in tqdm(range(1, 10)): 
    data = utils.load_data(PATH_TRAIN, index_)
    ## Fit treatment
    model_t = LogisticRegression()
    model_t.fit(data[cov], data[treatment])

    ## Fit outcome
    model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    model_y.fit(data[features], data[outcome])

    data['p1'] = model_t.predict_proba(data[cov])[:,1]
    data['p0'] = 1 - data['p1']

    ## Compute counterfactual outcome with no treatment
    data_pos = data.copy()
    data_pos[treatment] = 1
    data['cf1'] = model_y.predict(data_pos[features])

    ## Compute counterfactual outcome with treatment
    data_neg = data.copy()
    data_neg[treatment] = 0
    data['cf0'] = model_y.predict(data_neg[features])

    data['ips_weight'] = (data[treatment] / data['p1'] + (1 - data[treatment]) /
                          (1 - data['p1']))
    
    data['w0'] = data['ips_weight']*data[treatment]
    data['w1'] = data['ips_weight']*(1 - data[treatment])
    
    delta, loss = optimization(data)
    delta_r = delta.numpy()
    print(delta_r.shape)
    delta_seq.append(delta_r)
    print("Data index {}. Loss {:.2f}".format(index_, loss.numpy()))
 

 11%|█         | 1/9 [04:49<38:39, 289.88s/it]

(672,)
Data index 1. Loss 0.00


 22%|██▏       | 2/9 [04:54<23:50, 204.32s/it]

The performance reach MAE: 0.001. Cancelling the training at step 31
(672,)
Data index 2. Loss 0.00


 33%|███▎      | 3/9 [09:35<22:43, 227.29s/it]

...

(672,)
Data index 7. Loss 0.05


 89%|████████▉ | 8/9 [33:31<04:37, 277.51s/it]

(672,)
Data index 8. Loss 0.02


100%|██████████| 9/9 [38:27<00:00, 256.36s/it]

(672,)
Data index 9. Loss 0.26


In [21]:
np.array(delta_seq).shape

In [22]:
save('list_delta_.npy', delta_seq)






## Optimization for individual treatment effect

In [8]:
def incre_ps(delta, data):
    q1 = (delta * data['p1']) / (delta * data['p1'] + data['p0'])
    q1 = tf.math.abs(q1)
    a0 = (1-q1)*data['w0']*(data['cf0'] - data[outcome])
    a1 = q1*data['w1']*(data['cf1'] - data[outcome])    
    influence = a1 - a0
    return influence

def optimization(data):
    threhold = tf.constant([0.01])
    '''
    delta = tf.Variable(
        tf.random.uniform([data.shape[0],], 
                          minval=1, 
                          maxval=100, 
                          dtype=tf.dtypes.float32), 
                          trainable = True)
    '''
    delta = tf.Variable(tf.random.normal(
        [data.shape[0],], 
        mean=0, 
        stddev=10, 
        dtype=tf.dtypes.float32, 
        seed=1, 
        name='delta'
    ), trainable = True)
    
    true_effect = data['mu1'] - data['mu0']
    
    for i in range(100000):
        with tf.GradientTape() as tape:
            influence = incre_ps(delta, data)
            loss = tf.keras.losses.MAE(true_effect, influence)
            d_delta = tape.gradient(loss, delta)
            opt = tf.keras.optimizers.Adam(learning_rate=0.001)
            opt.apply_gradients(zip([d_delta], [delta]))
            if i % 1000 == 0:
                print("Epoch {}. Loss {:.5f}".format(i, loss.numpy()))
            if tf.math.less(loss, threhold):
                print("The performance reach MAE: 0.001. Cancelling the training at step {}".format(i))
                break
    return delta, loss

In [9]:
PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.train.npz"
PATH_TEST = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.test.npz"

delta_seq = []
for index_ in tqdm(range(13, 14)): 
    data = utils.load_data(PATH_TRAIN, index_)
    ## Fit treatment
    model_t = LogisticRegression()
    model_t.fit(data[cov], data[treatment])

    ## Fit outcome
    model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    model_y.fit(data[features], data[outcome])

    data['p1'] = model_t.predict_proba(data[cov])[:,1]
    data['p0'] = 1 - data['p1']

    ## Compute counterfactual outcome with no treatment
    data_pos = data.copy()
    data_pos[treatment] = 1
    data['cf1'] = model_y.predict(data_pos[features])

    ## Compute counterfactual outcome with treatment
    data_neg = data.copy()
    data_neg[treatment] = 0
    data['cf0'] = model_y.predict(data_neg[features])

    data['ips_weight'] = (data[treatment] / data['p1'] + (1 - data[treatment]) /
                          (1 - data['p1']))
    
    data['w0'] = data['ips_weight']*data[treatment]
    data['w1'] = data['ips_weight']*(1 - data[treatment])
    
    delta, loss = optimization(data)
    delta_r = delta.numpy()
    delta_seq.append(delta_r)
    print("Data index {}. Loss {:.2f}".format(index_, loss.numpy()))

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 0. Loss 128.21106
Epoch 1000. Loss 35.25104
Epoch 2000. Loss 26.33487
Epoch 3000. Loss 22.03906
Epoch 4000. Loss 19.51452
Epoch 5000. Loss 17.79730
Epoch 6000. Loss 16.47934
Epoch 7000. Loss 15.40867
Epoch 8000. Loss 14.58724
Epoch 9000. Loss 13.90437
Epoch 10000. Loss 13.29880
Epoch 11000. Loss 12.75287
Epoch 12000. Loss 12.24901
Epoch 13000. Loss 11.83835
Epoch 14000. Loss 11.50295
Epoch 15000. Loss 11.18480
Epoch 16000. Loss 10.90050
Epoch 17000. Loss 10.66358
Epoch 18000. Loss 10.45095
Epoch 19000. Loss 10.25655
Epoch 20000. Loss 10.07695
Epoch 21000. Loss 9.91892
Epoch 22000. Loss 9.77304
Epoch 23000. Loss 9.63611
Epoch 24000. Loss 9.50814
Epoch 25000. Loss 9.38981
Epoch 26000. Loss 9.27811
Epoch 27000. Loss 9.17255
Epoch 28000. Loss 9.07324
Epoch 29000. Loss 8.97934
Epoch 30000. Loss 8.89013
Epoch 31000. Loss 8.80546
Epoch 32000. Loss 8.72501
Epoch 33000. Loss 8.64893
Epoch 34000. Loss 8.57677
Epoch 35000. Loss 8.50806
Epoch 36000. Loss 8.44242
Epoch 37000. Loss 8.37966
Epo

100%|██████████| 1/1 [09:38<00:00, 578.35s/it]

Data index 13. Loss 6.87


In [12]:
save('individual_list_delta_.npy', delta_seq)

## Optimization for individual treatment effects using genetic algorithm

In [6]:
def incre_ps(delta, data):
    q1 = (delta * data['p1']) / (delta * data['p1'] + data['p0'])
    q1 = abs(q1)
    a0 = (1-q1)*data['w0']*(data['cf0'] - data[outcome])
    a1 = q1*data['w1']*(data['cf1'] - data[outcome])    
    influence = a1 - a0
    return influence


def fitness_function(delta, solution_idx):
    influence = incre_ps(delta, data)
    mae = -np.sqrt(np.mean((true_effect - influence)**2))
    return np.mean(mae)



In [7]:
PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.train.npz"
data = utils.load_data(PATH_TRAIN, 1)

## Fit treatment
model_t = LogisticRegression()
model_t.fit(data[cov], data[treatment])

## Fit outcome
model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
model_y.fit(data[features], data[outcome])

data['p1'] = model_t.predict_proba(data[cov])[:,1]
data['p0'] = 1 - data['p1']

## Compute counterfactual outcome with no treatment
data_pos = data.copy()
data_pos[treatment] = 1
data['cf1'] = model_y.predict(data_pos[features])

## Compute counterfactual outcome with treatment
data_neg = data.copy()
data_neg[treatment] = 0
data['cf0'] = model_y.predict(data_neg[features])

data['ips_weight'] = (data[treatment] / data['p1'] + (1 - data[treatment]) /
                      (1 - data['p1']))

data['w0'] = data['ips_weight']*data[treatment]
data['w1'] = data['ips_weight']*(1 - data[treatment])


In [8]:
true_effect = data['mu1'] - data['mu0']

In [ ]:
import pygad
import numpy

"""
Given the following function:
    y = f(w1:w6) = w1x1 + w2x2 + w3x3 + w4x4 + w5x5 + 6wx6
    where (x1,x2,x3,x4,x5,x6)=(4,-2,3.5,5,-11,-4.7) and y=44
What are the best values for the 6 weights (w1 to w6)? We are going to use the genetic algorithm to optimize this function.
"""

#function_inputs = [4,-2,3.5,5,-11,-4.7] # Function inputs.
#desired_output = 44 # Function output.

function_inputs = np.random.rand(data.shape[0]) # Function inputs.

num_generations = 5000 # Number of generations.
num_parents_mating = 100 # Number of solutions to be selected as parents in the mating pool.

# To prepare the initial population, there are 2 ways:
# 1) Prepare it yourself and pass it to the initial_population parameter. This way is useful when the user wants to start the genetic algorithm with a custom initial population.
# 2) Assign valid integer values to the sol_per_pop and num_genes parameters. If the initial_population parameter exists, then the sol_per_pop and num_genes parameters are useless.
sol_per_pop = 2500 # Number of solutions in the population.
num_genes = len(function_inputs)

init_range_low = -100
init_range_high = 100

parent_selection_type = "sss" # Type of parent selection.
keep_parents = -1 # Number of parents to keep in the next population. -1 means keep all parents and 0 means keep nothing.

crossover_type = "single_point" # Type of the crossover operator.

# Parameters of the mutation operation.
mutation_type = "random" # Type of the mutation operator.
mutation_percent_genes = 10 # Percentage of genes to mutate. This parameter has no action if the parameter mutation_num_genes exists or when mutation_type is None.

last_fitness = 0
def callback_generation(ga_instance):
    global last_fitness
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution()[1]))
    print("Change     = {change}".format(change=ga_instance.best_solution()[1] - last_fitness))
    print("\n")
    last_fitness = ga_instance.best_solution()[1]

# Creating an instance of the GA class inside the ga module. Some parameters are initialized within the constructor.
ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating, 
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop, 
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       callback_generation=callback_generation)

# Running the GA to optimize the parameters of the function.
ga_instance.run()
